# References

1. https://www.kaggle.com/dschettler8845/visual-in-depth-eda-vinbigdata-competition-data

2. https://www.kaggle.com/trungthanhnguyen0502/eda-vinbigdata-chest-x-ray-abnormalities

3. https://www.kaggle.com/bhallaakshit/dicom-wrangling-and-enhancement

4. https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-train

Thanks for above great works!

# Goal of this experiment
(Written in Korean, but I will edit in English asap)

* Because X-ray imaging devices cannot focus like optical lenses, The resulting image generally tends to be slightly blurred.
* Radiologists diagnose X-ray images with their own eyes, so it is determined that the black/white ratio and shape (blood vessels, lungs) are the criteria, which are determined by the edge.
* Therefore, I think it would be helpful to emphasize this edge information on the image through pre-processing.
* Image enhancement was experimented using several basic methods
* And train yolo v5 model with enhanced images to see if this affects the actual performance.
* Also, preprocessed image datasets (abnormal classes) are also provided as a result.



In [ ]:
import numpy as np
import pandas as pd 
import os
import shutil
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
from skimage import exposure
from glob import glob

from scipy.io import wavfile
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GroupKFold


# Enhancement trial

In [ ]:
dataset_dir = '../input/vinbigdata-chest-xray-abnormalities-detection'

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data
        
    
def plot_img(img, size=(7, 7), is_rgb=True, title="", cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()


def plot_imgs(imgs, cols=4, size=7, is_rgb=True, title="", cmap='gray', img_size=(500,500)):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

# Origninal images

In [ ]:
dicom_paths = glob(f'{dataset_dir}/train/*.dicom')
imgs = [dicom2array(path) for path in dicom_paths[:8]]
plot_imgs(imgs)

# Invert

In [ ]:
invert = 256 - np.array(imgs)
plot_imgs(invert)

# filters
1) 3x3 High pass filter

In [ ]:
hf1 = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])

after_hf1 = [cv2.filter2D(img, -1, hf1) for img in imgs]
plot_imgs(after_hf1)

It seems worse than originals..
* noise
* appear wave patterns

# Histogram Equalization

In [ ]:
hist_eq = [cv2.equalizeHist(img) for img in imgs]
plot_imgs(hist_eq)

# CLAHE (Contrast Limited Adaptive Histogram Equalization)


In [ ]:
def clahe(image):
    clahe = cv2.createCLAHE(
        clipLimit = 2., 
        tileGridSize = (10, 10)
    )
    
    image = clahe.apply(image) 
    #image = tf.expand_dims(image, axis = 2)
    
    return image

In [ ]:
clahe_ = [clahe(img) for img in imgs]
plot_imgs(clahe_)

It looks best.

# Data Settings

In [ ]:
dim = 512 #1024, 256, 'original'
test_dir = f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/test'
weights_dir = '/kaggle/input/vinbigdata-cxr-ad-yolov5-14-class-train/yolov5/runs/train/exp/weights/best.pt'

In [ ]:
train_df = pd.read_csv(f'../input/vinbigdata-{dim}-image-dataset/vinbigdata/train.csv')
train_df.head()

In [ ]:
train_df['image_path'] = f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/train/'+train_df.image_id+('.png' if dim!='original' else '.jpg')
train_df.head()

In [ ]:
train_df = train_df[train_df.class_id!=14].reset_index(drop = True)

Data split

In [ ]:
fold = 4
gkf  = GroupKFold(n_splits = 5)
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df, groups = train_df.image_id.tolist())):
    train_df.loc[val_idx, 'fold'] = fold
val_df = train_df[train_df['fold']==4]
val_df.head()

Train : 3515, val : 879

In [ ]:
train_files = []
val_files   = []
val_files += list(train_df[train_df.fold==fold].image_path.unique())
train_files += list(train_df[train_df.fold!=fold].image_path.unique())
len(train_files), len(val_files)

In [ ]:
os.makedirs('/kaggle/working/vinbigdata/labels/train', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/labels/val', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/images/train', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/images/val', exist_ok = True)
label_dir = '/kaggle/input/vinbigdata-yolo-labels-dataset/labels'
for file in train_files:
    shutil.copy(file, '/kaggle/working/vinbigdata/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/vinbigdata/labels/train')
    
for file in val_files:
    shutil.copy(file, '/kaggle/working/vinbigdata/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/vinbigdata/labels/val')
    
val_dir = f'/kaggle/working/vinbigdata/images/val'

# Setting for enhanced images

* Enhance only train images (not for val images)

In [ ]:
os.makedirs('/kaggle/working/vinbigdata_processed/images/train', exist_ok = True)
shutil.copytree('/kaggle/working/vinbigdata/labels/train','/kaggle/working/vinbigdata_processed/labels/train')
shutil.copytree('/kaggle/working/vinbigdata/labels/val','/kaggle/working/vinbigdata_processed/labels/val')
shutil.copytree('/kaggle/working/vinbigdata/images/val','/kaggle/working/vinbigdata_processed/images/val')

In [ ]:
for file in train_files:
    shutil.copy(file, '/kaggle/working/vinbigdata/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/vinbigdata_processed/labels/train')
    

# Save Enhancemented Images with .png format

In [ ]:
path = '/kaggle/working/vinbigdata/images/train'
save_path = '/kaggle/working/vinbigdata_processed/images/train'
images = os.listdir(path)

for image in images:
    img_path = os.path.join(path, image)
    ori_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    dst_img = clahe(ori_img)

    cv2.imwrite(os.path.join(save_path, image), dst_img)

    


Save train images after clahe

In [ ]:
print(len(os.listdir(save_path)))
sample = cv2.imread(os.path.join(save_path, images[0]))
plt.imshow(sample)

# Setups for training yolo v5

In [ ]:
class_ids, class_names = list(zip(*set(zip(train_df.class_id, train_df.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes

In [ ]:
from os import listdir
from os.path import isfile, join
import yaml

cwd = '/kaggle/working/'

with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('/kaggle/working/vinbigdata_processed/images/train/*'):
        f.write(path+'\n')
            
with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('/kaggle/working/vinbigdata_processed/images/val/*'):
        f.write(path+'\n')

data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'val.txt' ),
    nc    = 14,
    names = classes
    )

with open(join( cwd , 'vinbigdata.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( cwd , 'vinbigdata.yaml'), 'r')
print('\nyaml:')
print(f.read())

In [ ]:
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5') # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 30 --data /kaggle/working/vinbigdata.yaml --weights yolov5x.pt --cache


# Inference

In [ ]:
!python detect.py --weights '/kaggle/working/yolov5/runs/train/exp/weights/best.pt'\
--img 640\
--conf 0.15\
--iou 0.5\
--source /kaggle/working/vinbigdata_processed/images/val\
--exist-ok

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import random
import cv2
from glob import glob
from tqdm import tqdm

files = glob('runs/detect/exp/*')

for _ in range(1):
    row = 4
    col = 4
    grid_files = files[:16]
    images     = []
    for image_path in tqdm(grid_files):
        img          = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        images.append(img)

    fig = plt.figure(figsize=(col*5, row*5))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(col, row),  # creates 2x2 grid of axes
                     axes_pad=0.05,  # pad between axes in inch.
                     )

    for ax, im in zip(grid, images):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

# Compare with non-clahe image train models

In [ ]:
weights_dir = '/kaggle/input/vinbigdata-cxr-ad-yolov5-14-class-train/yolov5/runs/train/exp/weights/best.pt'
# train 30 epochs
# val mAP_0.5 : 0.31919
# val mAP_0.5:0.95 : 0.14161

In [ ]:
!python detect.py --weights $weights_dir\
--img 640\
--conf 0.15\
--iou 0.5\
--source /kaggle/working/vinbigdata_processed/images/val\
--save-txt --save-conf --exist-ok


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import random
import cv2
from glob import glob
from tqdm import tqdm

files = glob('runs/detect/exp/*')

for _ in range(1):
    row = 4
    col = 4
    grid_files = files[:16]
    images     = []
    for image_path in tqdm(grid_files):
        img          = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        images.append(img)

    fig = plt.figure(figsize=(col*5, row*5))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(col, row),  # creates 2x2 grid of axes
                     axes_pad=0.05,  # pad between axes in inch.
                     )

    for ax, im in zip(grid, images):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

# Compare with GT labels